Get articles' data and metadata from MongoDB

In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

## Connect to database

In [2]:
client = MongoClient('mongodb://10.237.26.159', 27017)
database = client['media-db']
collections = {}
collections['aadhar'] = database['aadhar-all']
collections['gst'] = database['gst-all']
collections['demonetization'] = database['demonetization-all']
collections['farmers'] = database['farmers-all']

## Create histograms

In [3]:
articles = {} # articles for each policy-event
dateMap = {}  # no. of articles each day
textMap={}
f = open('all-articles.text','w')
# f = open('aadhar-articles.text','w')
for policy_name in collections:
    articles[policy_name] = collections[policy_name].find({})

for policy_name in articles:
    dateMap[policy_name] = {}
    textMap[policy_name] = []
    
    for article in articles[policy_name]:
        ptime = pd.to_datetime( article['publishedDate'] )
        ptext = article['text']
        f.write(ptext+'\n')
        
        if ptime in dateMap[policy_name]:
            dateMap[policy_name][ptime] += 1
        else:
            dateMap[policy_name][ptime] = 1
        
    dateMap[policy_name] = pd.DataFrame([dateMap[policy_name]]).transpose()
f.close()

In [4]:
for policy_name in dateMap:
    total = sum(dateMap[policy_name].values)[0]
    variance = dateMap[policy_name].var(axis = 0)[0]
    mean = dateMap[policy_name].mean(axis = 0)[0]
    print( policy_name + ':\tTotal -> ', total, '; Variance -> ', variance, '; Mean -> ', mean )
    dateMap[policy_name].plot(figsize=(15, 6), grid = True, title=policy_name)

aadhar:	Total ->  13908 ; Variance ->  18.693403086272152 ; Mean ->  4.961826614341777
gst:	Total ->  22179 ; Variance ->  230.36921530984895 ; Mean ->  8.378919531545145
demonetization:	Total ->  22302 ; Variance ->  395.66020964218006 ; Mean ->  7.770731707317073
farmers:	Total ->  85486 ; Variance ->  129.19153533759177 ; Mean ->  27.399358974358975
